

**Loading Saved Model**

In [0]:
from keras.models import load_model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from keras.optimizers import SGD
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from random import shuffle
#model = load_model('model-accuracy-99%.h5')
test_dir = r'/content/action/'
test1_dir = r'E:\Nikhil\python\videoclass\videoclass\test\test1'
test2_dir=r'E:\Nikhil\python\machinelearningex\videoclass\videoclass\test'
train1_data=r'/content/action/train'
validation1_data=r'/content/action/test'
model=Sequential()
model=load_model('model4-imggen.h5')
lrate=0.01
epochs1=50
decay=lrate/epochs1
sgd = SGD(lr=lrate,momentum=0.9,decay=decay,nesterov=False)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
test_datagen= ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(120,120),
    classes=['testing'],
    shuffle=False,
    batch_size=10000,
    )
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    train1_data,
    target_size=(120,120),
    batch_size=32,
    shuffle=False,
    classes=['boxing1','handclapping','handwaving','jogging'],
    )
test_generator.reset()
train_generator.reset()
#print(test_generator.filenames)
#model1=model.predict_generator(test_generator,1200)
test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = test_datagen.flow_from_directory(
    validation1_data,
    target_size=(120,120),
    classes=['boxing','handclapping','handwaving','jogging'],
    batch_size=12,
    )




pred= model.predict_generator(test_generator, 1)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (validation_generator.class_indices)
labels2 = dict((v,k) for k,v in labels.items())
predictions = [labels2[k] for k in predicted_class_indices]
#print(predicted_class_indices)
#print (labels)
            
#print (predictions)

dt={}
for i in range(len(test_generator.filenames)):
        
    name=test_generator.filenames[i].split('\\')[-1]
    dt[name]=predictions[i]
    

print(dt)
    


    



In [0]:
!git clone https://github.com/Nikhilhc/action.git

Cloning into 'action'...
remote: Enumerating objects: 119383, done.
remote: Counting objects: 100% (119383/119383), done.
remote: Compressing objects: 100% (119379/119379), done.
remote: Total 119383 (delta 1), reused 119383 (delta 1), pack-reused 0
Receiving objects: 100% (119383/119383), 858.12 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (126698/126698), done.


**Packages Required For Training Model**

In [0]:

import keras
import numpy as np
from tqdm import tqdm
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from keras.preprocessing.image import ImageDataGenerator
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
from random import shuffle
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.models import Sequential
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.layers import GlobalMaxPooling2D

**Creating Training set and Testing dataset using tqdm**

In [0]:

train1_data = '/content/action/action2/humanactivity/train'
validation1_data='/content/action/action2/humanactivity/val'
IMG_SIZE=120
IMG_SIZE1=120
LR = 1e-3

def create_label(image_name):
    word_label=image_name.split('_')[-2]
    if word_label=='boxing':
        return 'boxing'
    if word_label =='handclapping':
        return 'handclapping'
    if word_label =='handwaving':
        return 'handwaving'
    if word_label == 'jogging':
        return 'jogging'
    if word_label == 'running':
        return 'running'
    if word_label == 'walking':
        return 'walking'
    

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(train1_data)):
        path = os.path.join(train1_data,img)
        img_data = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img_data=cv2.resize(img_data,(IMG_SIZE,IMG_SIZE1))
        training_data.append([np.array(img_data),create_label(img)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

def create_test_data():
    test_data = []
    for img in tqdm(os.listdir(validation1_data)):
        path = os.path.join(validation1_data,img)
        img_data = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img_data=cv2.resize(img_data,(IMG_SIZE,IMG_SIZE1))
        test_data.append([np.array(img_data),create_label(img)])
    shuffle(test_data)
    np.save('test_data.npy',test_data)
    return test_data

train_data = create_train_data()
test_data=create_test_data()

train = train_data[:-1]
test = train_data[-1200:]

X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE1,1)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE1,1)
y_test =[i[1] for i in test]

X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
print(y_test)

encoder=LabelEncoder()
encoder.fit(y_test)
y_test=encoder.transform(y_test)
encoder.fit(y_train)
y_train=encoder.transform(y_train)
print(y_test)

X_train=X_train/255.0
X_test=X_test/255.0
y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)
num_classes=6
print(y_test)


**Training The Model**

In [0]:

import keras
import numpy as np
#from parse import load_data
from tqdm import tqdm
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from keras.preprocessing.image import ImageDataGenerator
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
from random import shuffle
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.models import Sequential
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.layers import GlobalMaxPooling2D


train1_data = '/content/action/action2/humanactivity/train'
validation1_data='/content/action/action2/humanactivity/val'
IMG_SIZE=120
IMG_SIZE1=120
LR = 1e-3

model=Sequential()
model.add(Conv2D(32,3,3,input_shape=(IMG_SIZE,IMG_SIZE1,3),border_mode='same',
                        activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32,3,3,activation='relu',border_mode='same'
                        ))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering="th"))

model.add(Conv2D(64,3,3,activation='relu',border_mode='same'
                        ))
model.add(Dropout(0.2))
model.add(Conv2D(64,3,3,activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering="th"))


model.add(Conv2D(128,3,3,activation='relu',border_mode='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128,3,3,activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering="th"))



model.add(Flatten())


model.add(Dense(4,activation='softmax'))

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
epochs1=50
lrate=0.01
decay=lrate/epochs1
sgd = SGD(lr=lrate,momentum=0.9,decay=decay,nesterov=False)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
checkpoint=keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1)
callbacks_lst=[checkpoint]
print(model.summary())
#model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs1,batch_size=16,verbose=2)
#scores = model.evaluate(X_test,y_test,verbose=0)
#print("Accuracy:%.2f%%"%(scores[1]*100))
shift=0.2
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(
    train1_data,
    target_size=(IMG_SIZE, IMG_SIZE1),
    batch_size=128,
    classes=['boxing','handclapping','handwaving','walking'],
    )

validation_generator = test_datagen.flow_from_directory(
    validation1_data,
    target_size=(IMG_SIZE,IMG_SIZE1),
    classes=['boxing','handclapping','handwaving','walking'],
    batch_size=128,
    )

model.fit_generator(
    train_generator,
    steps_per_epoch=130000// 128, #nb_train_samples//batch_size
    epochs=25,
    validation_data=validation_generator,
    callbacks=callbacks_lst,
    validation_steps=2000/128)
    
model.save('model5-imggen.h5')


Using TensorFlow backend.
W0723 01:22:49.113529 140093030946688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0723 01:22:49.115188 140093030946688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0723 01:22:49.125541 140093030946688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0723 01:22:49.132407 140093030946688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0723 01:22:49.141719 140093030946688 deprecation_wrapper.py

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 120, 120, 32)      896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 120, 120, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 60, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 120, 60, 64)       9280      
_________________________________________________________________
dropout_2 (Dropout)          (None, 120, 60, 64)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 120, 60, 64)       36928     
__________

W0723 01:22:52.898376 140093030946688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Found 2000 images belonging to 4 classes.
Epoch 1/25
1015/1015 [==============================] - 762s 750ms/step - loss: 0.9502 - acc: 0.5753 - val_loss: 0.4755 - val_acc: 0.8205

Epoch 00001: val_acc improved from -inf to 0.82050, saving model to weights-improvement-01-0.82.hdf5
Epoch 2/25
1015/1015 [==============================] - 752s 740ms/step - loss: 0.4038 - acc: 0.8405 - val_loss: 0.2305 - val_acc: 0.9160

Epoch 00002: val_acc improved from 0.82050 to 0.91600, saving model to weights-improvement-02-0.92.hdf5
Epoch 3/25
1015/1015 [==============================] - 749s 738ms/step - loss: 0.2645 - acc: 0.8977 - val_loss: 0.1720 - val_acc: 0.9395

Epoch 00003: val_acc improved from 0.91600 to 0.93950, saving model to weights-improvement-03-0.94.hdf5
Epoch 4/25
1015/1015 [==============================] - 746s 735ms/step - loss: 0.1928 - acc: 0.9278 - val_loss: 0.1045 - val_acc: 0.9630

Epoch 00004: val_acc improved from 0.93950 to 0.96300, saving model to weights-improvement-04

**Converting Video into frames** 

In [0]:

import random,os
import cv2
import shutil
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
count = 0
for i in range(1,100):
    videoFile = r"/content/action1/KTHdataset/boxing/1i_%dv.avi"%i
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    
    x=1
    while True:
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret == True):
            filename =r"/content/action1/train/boxing/boxing_%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
        else:
            break
    cap.release()
    print ("Done!")





    



**Detecting Human In The Frames**

In [0]:
import cv2
import os
import shutil

video_src = 'pedestrians.avi'
count=0
for r in range(0,65000):
    e=r'/content/action/walking/walking_%d.jpg'%r
    f=r'/content/action/none1/walking_%d.jpg'%r
    img=cv2.imread(e)

    bike_cascade = cv2.CascadeClassifier('/content/project1/pedx_mcs.xml')
    bike_cascade1 = cv2.CascadeClassifier('/content/project1/pedxlower.xml')

        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bike = bike_cascade.detectMultiScale(img,1.009,2)
    bike1 = bike_cascade1.detectMultiScale(img,1.001,2)
    if bike==():
        pass
    elif bike.all()>0:
        count=count+1
    if bike1==():
        pass
    elif bike1.all()>0:
        count=count+1
    if bike==() and bike1==():
        shutil.move(e,f)
    print(count)

    

cv2.destroyAllWindows()